# Código Auxiliar para a Segunda Avaliação.

In [23]:
import pandas as pd
import sqlite3
from collections import defaultdict

As funções definidas abaixo, devem ser usada para acessar o banco de dados `bilheteria.db` e retornar os dados necessários para a resolução dos exercícios.


In [24]:
def carrega_tabela(database, tabela):
    '''
    Carrega uma tabela de um banco de dados 
    SQLite em um DataFrame do Pandas.
    Parâmetros:
    database: str - Caminho para o arquivo do banco de dados SQLite.
    tabela: str - Nome da tabela a ser carregada.
    '''
    with sqlite3.connect(database) as conn:
        query = f"SELECT * FROM {tabela}"
        df = pd.read_sql_query(query, conn)
    return df

In [25]:
def lista_tabelas(db_filename):
    '''
    Lista todas as tabelas em um banco de dados SQLite.
    Parâmetros:
    db_filename: str - Caminho para o arquivo do banco de dados SQLite.
    Retorna:
    DataFrame com os nomes das tabelas e suas contagens de linhas.
    '''
    # Connect to the SQLite database
    # conn = sqlite3.connect(db_filename)
    with sqlite3.connect(db_filename) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()

        table_row_counts = []
        for table in tables:
            table_name = table[0]
            query = f"SELECT COUNT(*) FROM {table_name};"
            cursor.execute(query)
            row_count = cursor.fetchone()[0]
            table_row_counts.append({"Table": table_name, "Row_Count": row_count})

        return pd.DataFrame(table_row_counts)

## Exemplos de uso das funções

In [26]:
lista_tabelas('bilheteria.db')

,Table,Row_Count
0,distribuidora,72
1,filme,515
2,grupo_exibidor,64
3,exibidor,180
4,complexo,683
5,sala,3231
6,sessao,1748363


# Integrantes da equipe

In [27]:
AUTORES = ['Angel Machado', 'Elainne Gutiérrez']

# Utilize a célula abaixo para visualizar os dados

Comente e descomente as tabelas que queira visualizar

In [28]:
#distribuidora = carrega_tabela("bilheteria.db", "distribuidora")
filme = carrega_tabela("bilheteria.db", "filme")
#grupo_exibidor = carrega_tabela("bilheteria.db", "grupo_exibidor")
#exibidor = carrega_tabela("bilheteria.db", "exibidor")
#complexo = carrega_tabela("bilheteria.db", "complexo")
#sala = carrega_tabela("bilheteria.db", "sala")
#sessao = carrega_tabela('bilheteria.db', 'sessao')

#print(distribuidora)
print(filme)
#print(filme.columns)
#print(grupo_exibidor)
#print(exibidor)
#print(complexo)
#print(sala)
#print(sessao)


      id                                 titulo_original  \
0      1                                   DURVAL DISCOS   
1      2                                       NOSSO LAR   
2      3                                         REMOÇÃO   
3      4                    HOJE EU QUERO VOLTAR SOZINHO   
4      5                          O HOMEM DE LAGOA SANTA   
..   ...                                             ...   
510  511             NCT NATION: TO THE WORLD IN CINEMAS   
511  512                            JOURNEY TO BETHLEHEM   
512  513                                       PRISCILLA   
513  514  SEVENTEEN TOUR 'FOLLOW' TO JAPAN: LIVE VIEWING   
514  515                                      THE CHOSEN   

                                             titulo_br         cpb_roe  \
0                                                 None  B0200001000000   
1                                                 None  B1001259400000   
2                                                 None  B

# Questão 1
1. Qual o total de bilheteria de todos os filmes, ou seja, o público que foi aos
filmes listados?

In [29]:
def questao_1(database: str):
    #carregamos a tabela necessária
    sessao = carrega_tabela(database, "sessao")
    #coluna do público
    total_bilheteria_dataframe = sessao.publico
    total_bilheteria = sum(total_bilheteria_dataframe)
    print(f"O total de bilheteria dos filmes listados foi: {total_bilheteria}")
    return total_bilheteria

questao_1("bilheteria.db")

O total de bilheteria dos filmes listados foi: 110153083


110153083

# Questão 2
2. Qual o filme de maior bilheteria em 2023, por país de origem?

In [30]:
def questao_2(database: str):
    #carregamos as tabelas necessárias com as funções
    filme = carrega_tabela(database, "filme") 
    sessao = carrega_tabela(database, "sessao")

    #a partir da tabela filme, selecionamos as colunas id, titulo_original e pais_origem
    id_titulo_pais = filme[
        [
            "id",
            "titulo_original",
            "pais_origem"
        ]
    ]
    
    #a partir da tabela sessao, selecionamos as colunas filme_id e publico
    filme_id_publico = sessao[
        [
            "filme_id",
            "publico"
        ]
    ]

    #criamos dicionários
    bilheteria_de_cada_filme_por_id = defaultdict(int)    #chave: id de cada filme, valor: total de bilheteria 
    id_de_cada_filme_por_pais = defaultdict(list)         #chave: país, valor: lista com os id's de cada filme desse país
    nome_de_cada_filme_por_id = defaultdict(str)          #chave: id de cada filme, valor: nome de cada filme
    filme_de_maior_bilheteria_por_pais = defaultdict(str) #chave: país, valor: filme com maior bilheteria desse país

    #colocamos cada país com seus filmes
    #colocamos cada filme_id com seu filme
    for index in id_titulo_pais.itertuples(index= False):
        filme_id = index.id
        titulo = index.titulo_original
        pais = index.pais_origem
        id_de_cada_filme_por_pais[pais].append(filme_id)
        nome_de_cada_filme_por_id[filme_id] = titulo
    
    #colocamos cada filme_id com seu público total
    for index in filme_id_publico.itertuples(index=False):
        filme_id = index.filme_id
        publico = index.publico
        bilheteria_de_cada_filme_por_id[filme_id] += publico

    #colocamos cada país com seu filme de maior bilheteria
    for cada_pais in id_de_cada_filme_por_pais.keys():
        bilheteria = 0
        filme = ""
        
        for cada_filme_id in id_de_cada_filme_por_pais[cada_pais]:
            if bilheteria_de_cada_filme_por_id[cada_filme_id] > bilheteria:
                bilheteria = bilheteria_de_cada_filme_por_id[cada_filme_id]
                filme = nome_de_cada_filme_por_id[cada_filme_id]

        filme_de_maior_bilheteria_por_pais[cada_pais] = filme

    filme_de_maior_bilheteria_por_pais = pd.DataFrame(list(filme_de_maior_bilheteria_por_pais.items()), columns=["pais","filme_de_maior_bilheteria"]).sort_values("pais").reset_index(drop=True)
    return filme_de_maior_bilheteria_por_pais

questao_2("bilheteria.db")

,pais,filme_de_maior_bilheteria
0,ALEMANHA,THE AMAZING MAURICE
1,ARGENTINA,BEM-VINDA VIOLETA
2,ARGÉLIA,LA DERNIÈRE REINE
3,AUSTRÁLIA,TALK TO ME
4,BELARUS (BIELORUSSIA),CATS IN THE MUSEUM
5,BRASIL,NOSSO SONHO
6,BÉLGICA,CLOSE
7,CANADÁ,PAW PATROL: THE MIGHTY MOVIE
8,CHILE,LA SOCIEDAD DE LA NIEVE
9,CHINA,OPERATION FORTUNE: RUSE DE GUERRE


# Questão 3
3. Crie um dataframe com as 100 cidades com maior bilheteria em 2023,
ordenadas de forma decrescente de bilheteria.

In [31]:
def questao_3(database: str):
    #carregamos as tabelas necessárias
    sala = carrega_tabela(database, "sala")
    complexo = carrega_tabela(database, "complexo")
    sessao = carrega_tabela(database, "sessao")

    """
    da tabela sessao pegamos as colunas sala_id e publico, com a função groupby, agrupamos objetos iguais na coluna sala_id.
    Ou seja, linhas com mesma id serão juntadas numa só e seus públicos, somados; usando a função sum. Assim teremos como resultado:
    sala_id, publico_total, reiniciamos o index, pois a função groupby coloca sala_id automaticamente como index e não queremos isso;
    queremos que sala_id seja também uma coluna"""
    sala_id_publico_total = sessao[["sala_id", "publico"]].groupby("sala_id").sum().reset_index()

    """
    da tabela sessao pegamos as colunas, id e complexo, que contêm o id de cada sala e o complexo ao qual elas pertencem
    renomeamos a coluna id para sala_id, para conseguir fazer merge com as duas tabelas  """
    sala_id_compexo = sala[["id","from_complexo"]].rename(columns={"id":"sala_id"})
    
    """
    processo análogo à tabela anterior, agora da tabela complexo selecionamos apenas id e municipio, renomeando o municipio por
    from_complexo para poder fazer merge com a tabela sala_id_complexo
    """
    complexo_e_cidade = complexo[["id","municipio"]].rename(columns={"id":"from_complexo"})

    """
    fazemos merge de cada tabela que temos nas colunas com mesmo nome, assim conseguimos uma tabela maior com a que conseguiremos
    a funcao groupby com cada cidade, agrupando as cidades semelhantes e somando o publico total de cada cidade
    """
    sala_id_publico_e_complexo = sala_id_publico_total.merge(sala_id_compexo, how="left",on="sala_id")
    sala_id_publico_complexo_e_cidade = sala_id_publico_e_complexo.merge(complexo_e_cidade, how="left",on="from_complexo")
    cidades_bilheteria_total = sala_id_publico_complexo_e_cidade[["municipio","publico"]].groupby("municipio").sum(numeric_only=True)

    #ordenamos em ordem decresente, pegamos as primeiras 100 linhas, reiniciamos o index e renomeamos as colunas
    cem100_cidades_com_mais_bilheteria = cidades_bilheteria_total.sort_values("publico", ascending=False).head(100).reset_index().rename(columns={"publico": "bilheteria_total","municipio":"cidade"})
    
    return cem100_cidades_com_mais_bilheteria
    
questao_3("bilheteria.db")

,cidade,bilheteria_total
0,SÃO PAULO,13310470
1,RIO DE JANEIRO,8496306
2,BRASÍLIA,3272437
3,BELO HORIZONTE,3009414
4,CURITIBA,2965114
...,...,...
95,ARAÇATUBA,247992
96,VITÓRIA DA CONQUISTA,246490
97,PONTA GROSSA,241142
98,LAURO DE FREITAS,240784


# Questão 4
4. Qual o filme com maior bilheteria em cada cidade? Retorne um dataframe
com as colunas CIDADE, FILME e BILHETERIA.

In [32]:
def questao_4(database):
    #carregamos as tabelas necessárias
    sessao = carrega_tabela(database, "sessao")
    filme = carrega_tabela(database, "filme")
    complexo = carrega_tabela(database, "complexo")
    sala = carrega_tabela(database, "sala")

    '''a partir daqui, renomearemos as colunas selecionadas das tabelas sala, complexo e filme 
    para que tenham o mesmo nome entre elas e as escolhidas da tabela sessao, a fim de poder fazer merge entre elas. '''
    #a partir da tabela sala, selecionamos as colunas id e from_complexo, renomeando-as para, respectivamente, sala_id e cidade_id
    sala_id__cidade__id = (
        sala[["id", "from_complexo"]]
        .rename(columns={"id":"sala_id","from_complexo":"cidade_id"})
        )
    
    #a partir da tabela complexo, selecionamos as colunas id e municipio, renomeando-as para, respectivamente, cidade_id e cidade
    cidade_id__cidade = (
        complexo[["id", "municipio"]]
        .rename(columns={"id":"cidade_id","municipio":"cidade"})
        )

    #a partir da tabela sessao, selecionamos as colunas sala_id, filme_id e publico
    #agrupamos por sala_id e filme_id semelhantes, somando o público total de cada sala para cada filme
    sala_id__filme_id__publico_total = (
        sessao[["filme_id","sala_id","publico"]]
        .groupby(["sala_id","filme_id"])
        .sum()
        .reset_index()
        )
    
    #a partir da tabela filme, selecionamos as colunas id e titulo_original, renomeando a primeira para filme_id
    filme_id__titulo_original = (
        filme[["id", "titulo_original"]]
        .rename(columns={"id":"filme_id"})
        )
    
    #merges
    sala_id__filme_id__publico_total__titulo = (
        sala_id__filme_id__publico_total
        .merge(filme_id__titulo_original, how="left", on="filme_id")
        )
    
    sala_id__cidade_id__cidade = (
        sala_id__cidade__id
        .merge(cidade_id__cidade, how="left", on="cidade_id")
        )

    sala_id__titulo_original__publico_total__cidade = (
        sala_id__filme_id__publico_total__titulo[["sala_id","titulo_original","publico"]]
        .merge(sala_id__cidade_id__cidade[["sala_id","cidade"]], how="left", on="sala_id")
        )
    
    cidade__filme__bilheteria = (
        sala_id__titulo_original__publico_total__cidade[["cidade","titulo_original","publico"]] #selecionamos colunas que nos interessam
        .sort_values(by=["cidade","publico"], ascending=[True,False]) #organizamos as cidades em ordem alfabética e o público de cada cidade em ordem decresente
        .drop_duplicates(subset="cidade") #este método apaga todas as linhas que tiverem uma cidade que já apareceu antes
        .reset_index(drop=True) #apagamos o antigo index bagunçado e adicionamos outro organizado
        .rename(columns={"cidade":"CIDADE","titulo_original":"FILME","publico":"BILHETERIA"}) #renomeamos as colunas para os nomes pedidos pela questão
        )

    return  cidade__filme__bilheteria

questao_4("bilheteria.db")


,CIDADE,FILME,BILHETERIA
0,ACARAÚ,BARBIE,3367
1,AGUAÍ,BARBIE,2140
2,ALAGOINHAS,BARBIE,6264
3,ALFENAS,BARBIE,3785
4,ALTAMIRA,BARBIE,2998
...,...,...,...
340,VOLTA REDONDA,BARBIE,6687
341,VOTORANTIM,AVATAR: THE WAY OF WATER,8070
342,VÁRZEA GRANDE,FAST X,6860
343,VÁRZEA PAULISTA,BARBIE,4107


# Questão 5
5. Quais as cidades com as maiores bilheterias para filmes brasileiros?
Retorne um dataframe com as colunas CIDADE, BILHETERIA_BR,
BILHETERIA_ESTRANGEIRA.

In [33]:
def questao_5(database):
    #carregamos as tabelas necessárias
    sessao = carrega_tabela(database, "sessao")
    filme = carrega_tabela(database, "filme")
    complexo = carrega_tabela(database, "complexo")
    sala = carrega_tabela(database, "sala")

    '''a partir daqui, renomearemos as colunas selecionadas das tabelas sala, complexo e filme 
    para que tenham o mesmo nome entre elas e as escolhidas da tabela sessao, a fim de poder fazer merge entre elas. '''
    #a partir da tabela sala, selecionamos as colunas id e from_complexo, renomeando-as para, respectivamente, sala_id e cidade_id
    sala_id__cidade__id = (
        sala[["id", "from_complexo"]]
        .rename(columns={"id":"sala_id","from_complexo":"cidade_id"})
        )
    
    #a partir da tabela complexo, selecionamos as colunas id e municipio, renomeando-as para, respectivamente, cidade_id e cidade
    cidade_id__cidade = (
        complexo[["id", "municipio"]]
        .rename(columns={"id":"cidade_id","municipio":"cidade"})
        )
    
    #a partir da tabela sessao, selecionamos as colunas sala_id, filme_id e publico
    #agrupamos esta última tabela por sala_id e filme_id semelhantes, somando o público total de cada sala para cada filme
    sala_id__filme_id__publico_total = (
        sessao[["filme_id","sala_id","publico"]]
        .groupby(["sala_id","filme_id"])
        .sum()
        .reset_index()
        )
    
    #a partir da tabela filme, selecionamos as colunas id, titulo_original e pais_origem, renomeando a primeira para filme_id
    filme_id__titulo_original__pais_origem = (
        filme[["id", "titulo_original", "pais_origem"]]
        .rename(columns={"id":"filme_id"})
        )
    
    #merges
    sala_id__filme_id__publico_total__titulo = (
        sala_id__filme_id__publico_total
        .merge(filme_id__titulo_original__pais_origem, how="left", on="filme_id")
        )
    
    sala_id__cidade_id__cidade = (
        sala_id__cidade__id
        .merge(cidade_id__cidade, how="left", on="cidade_id")
        )

    sala_id__titulo_original__publico_total__pais_origem__cidade = (
        sala_id__filme_id__publico_total__titulo[["sala_id","titulo_original","publico","pais_origem"]]
        .merge(sala_id__cidade_id__cidade[["sala_id","cidade"]], how="left", on="sala_id")
        )
    
    #agrupamos por cidade e pais_origem, somando o público total de cada cidade por país que lançou os filmes citados no database
    cidade__pais_origem__publico = (
        sala_id__titulo_original__publico_total__pais_origem__cidade[["cidade", "pais_origem","publico"]]
        .groupby(["cidade","pais_origem"])
        .sum("publico")
        .reset_index()
        )
    
    #da tabela criada imediatamente acima, criamos esta aqui, escolhendo apenas as linhas cujos públicos por cidade sejam referentes a filmes brasileiros
    #resetamos o index
    #renomeamos as colunas para os nomes pedidos pela questão
    cidade__bilheteria_br = (
        cidade__pais_origem__publico
        .loc[cidade__pais_origem__publico["pais_origem"] == "BRASIL"]
        [["cidade","publico"]]
        .reset_index(drop=True)
        .rename(columns={"cidade":"CIDADE","publico":"BILHETERIA_BR"})
        
        )
    
    #aqui, pelo mesmo processo, criamos uma tabela escolhendo as linhas cujos públicos por cidade sejam referentes a filmes estrangeiros
    cidade__bilheteria_estrangeira = (
        cidade__pais_origem__publico
        .loc[cidade__pais_origem__publico["pais_origem"] != "BRASIL"]
        [["cidade","publico"]]
        .groupby("cidade")
        .sum("publico")
        .reset_index()
        .rename(columns={"cidade":"CIDADE","publico":"BILHETERIA_ESTRANGEIRA"})
    )

    #merge das tabelas sobre bilheterias brasileira e estrangeira
    cidade__bilheteria_br__bilheteria_estrangeira = (
        cidade__bilheteria_br.merge(cidade__bilheteria_estrangeira, how="right", on="CIDADE")
    )

    cidade__bilheteria_br__bilheteria_estrangeira["BILHETERIA_BR"] = (
        cidade__bilheteria_br__bilheteria_estrangeira["BILHETERIA_BR"]
        .fillna(0) #preenche valores nulos com 0
        .astype("Int64") #converte para tipo long-int, que permite realizar cálculos com valores inteiros de maior tamanho que o suportado pelo tipo int32 ou int.
    )

    #retorna o DataFrame com a bilheteria brasileira em ordem decrescente e com o index reiniciado
    return cidade__bilheteria_br__bilheteria_estrangeira.sort_values(ascending=False, by="BILHETERIA_BR").reset_index(drop=True)
questao_5("bilheteria.db")

,CIDADE,BILHETERIA_BR,BILHETERIA_ESTRANGEIRA
0,RIO DE JANEIRO,546789,7949517
1,SÃO PAULO,403251,12907219
2,SALVADOR,166624,2304396
3,BELO HORIZONTE,98746,2910668
4,RECIFE,84134,2248439
...,...,...,...
337,LINS,0,18654
338,ASSIS,0,23569
339,JABOTICABAL,0,16460
340,ITAPEVA,0,19472
